In [1]:
import pandas as pd
data = pd.read_csv("dane.csv", delimiter=";", names=['From', 'To', 'Date', 'Cost'])

In [2]:
# data cleaning
data["To"] = data["To"].str.strip()
data['year'] = [int(i[0].split("-")[0]) for i in data['Date'].str.split()]
data['month'] = [int(i[0].split("-")[1]) for i in data['Date'].str.split()]
data['day'] = [int(i[0].split("-")[2]) for i in data['Date'].str.split()]
data['hour'] = [int(i[1].split(":")[0]) for i in data['Date'].str.split()]
data['minute'] = [int(i[1].split(":")[1]) for i in data['Date'].str.split()]
data['second'] = [int(i[1].split(":")[2]) for i in data['Date'].str.split()]
data = data.drop(["Date"], axis=1)

In [3]:
# LabelEncoder -> changes categorical data into numerical
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
labels = data['To'].tolist()
labels = labels + data['From'].tolist()
labels = list(set(labels))
le.fit(labels)
data['From_edge'] = le.transform(data['From'])
data['To_edge'] = le.transform(data['To'])

In [4]:
# data from customer (tak tak powinno byc city... nie chcialem juz zmieniac)
max_cost = 300
max_cities = 3
time_in_citie_in_days = 2
start_citie = "KRK"
year = 2019
month = 1
day = 10

In [5]:
data.head()

,From,To,Cost,year,month,day,hour,minute,second,From_edge,To_edge
0,KBP,WRO,36,2019,1,10,16,10,0,175,390
1,KBP,VNO,38,2019,1,10,10,25,0,175,384
2,KBP,VIN,41,2019,1,10,21,25,0,175,379
3,KBP,KRK,41,2019,1,10,21,40,0,175,186
4,KBP,CPH,42,2019,1,10,21,50,0,175,78


In [6]:
# create dataframe only with start city and all the connections from it
stage_1 = data.loc[(data['From'] == 'KRK') & (data['year'] == year) & (data['month'] == month) & (data['day'] == day) & (data['Cost'] <= max_cost)]

In [7]:
# create rest dataframes (with transit cities)
stages = [stage_1]
for stage in range(max_cities-1):
    day = day+time_in_citie_in_days
    stages.append(data.loc[(data['From'].isin(stages[-1]['To'].values)) & (data['To'] != start_citie) & (data['year'] == year) & (data['month'] == month) & (data['day'] == day) & (data['Cost'] <= max_cost)])
stages.append(data.loc[(data['From'].isin(stages[-1]['To'].values)) & (data['To'] == start_citie) & (data['year'] == year) & (data['month'] == month) & (data['day'] == day+time_in_citie_in_days) & (data['Cost'] <= max_cost)])

In [8]:
# make correction to cities numbers (ogolnie powinno byc bardziej From_vertex, To_vertex)
import numpy as np
tmp = 0
err = 0
for i, stage in enumerate(stages):
    if i != 0:
        err = tmp - np.max(stages[i-1]['From_edge'])
        stage['From_edge'] = le.transform(stage['From'].tolist())
        stage['From_edge'] += (np.max(stages[i-1]['From_edge']) + 1 + err)
        stage['To_edge'] = le.transform(stage['To'].tolist())
        stage['To_edge'] += (np.max(stages[i-1]['To_edge']) + 1)
        tmp = np.max(stages[i-1]['To_edge'])
        stages[i] = stage
    else:
        stage['From_edge'] = 0
        stage['To_edge'] = le.transform(stage['To'].tolist())
        stage['To_edge'] += 1
        stages[i] = stage

/home/amadeusz/anaconda3/envs/py35/lib/python3.5/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/amadeusz/anaconda3/envs/py35/lib/python3.5/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/home/amadeusz/anaconda3/envs/py35/lib/python3.5/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

In [9]:
# set index numbers as link numbers
length = 0
for i, stage in enumerate(stages):
    if i == 0:
        stage.index = stage.reset_index().drop(['index'], axis=1).index + 1
        length = len(stage)
    else:
        stage.index = stage.reset_index().drop(['index'], axis=1).index + 1 + length
        length = length + len(stage)

Save output as data.dat file

In [10]:
with open("data.dat", "a") as f:
    f.write("""
data;

param V_count := {V_count};
param E_count := {E_count};
param D_count := {D_count};

param : h  s t :=
 1      1 {s} {t}
;
        """.format(V_count=stages[-1]['To_edge'].iloc[0], 
                   E_count=stages[-1].tail(1).index.values[0], 
                   D_count=1, 
                   s=0, 
                   t=stages[-1]['To_edge'].iloc[0]))

In [11]:
with open("data.dat", "a") as f:
     f.write("""
param : A :=
""")

In [12]:
with open("data.dat", "a") as f:
    for stage in stages:
        for val, index in zip(stage.values, stage.index.values):
            f.write("""
 {link_number},{to_edge_number}    1
""".format(link_number=index, 
           to_edge_number=val[-2]))

In [13]:
with open("data.dat", "a") as f:
     f.write("""
;
""")

In [14]:
with open("data.dat", "a") as f:
     f.write("""
param : B :=
""")

In [15]:
with open("data.dat", "a") as f:
    for stage in stages:
        for val, index in zip(stage.values, stage.index.values):
            f.write("""
 {link_number},{to_edge_number}    1
""".format(link_number=index, 
           to_edge_number=val[-1]))

In [16]:
with open("data.dat", "a") as f:
     f.write("""
;
""")

In [17]:
with open("data.dat", "a") as f:
     f.write("""
param : KSI :=
""")

In [18]:
with open("data.dat", "a") as f:
    for stage in stages:
        for val, index in zip(stage.values, stage.index.values):
            f.write("""
 {link_number}    {cost}
""".format(link_number=index, 
           cost=val[2]))

In [19]:
with open("data.dat", "a") as f:
     f.write("""
;
""")

In [20]:
with open("data.dat", "a") as f:
     f.write("""
end;
""")

In [21]:
stages[0].loc[2]

From          KRK
To            SXF
Cost           11
year         2019
month           1
day            10
hour            9
minute         40
second          0
From_edge       0
To_edge       338
Name: 2, dtype: object

In [22]:
stages[1].loc[534]

From          SXF
To            VNO
Cost           15
year         2019
month           1
day            12
hour           14
minute         35
second          0
From_edge     338
To_edge       774
Name: 534, dtype: object

In [23]:
stages[2].loc[2913]

From          VNO
To            NYO
Cost           12
year         2019
month           1
day            14
hour            7
minute          5
second          0
From_edge     774
To_edge      1046
Name: 2913, dtype: object

In [24]:
stages[3].loc[3183]

From          NYO
To            KRK
Cost           11
year         2019
month           1
day            16
hour            9
minute          5
second          0
From_edge    1046
To_edge      1371
Name: 3183, dtype: object